In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d


### To Do list (medmont)


* 读取数据 ✔
* 填充为矩阵 ✔
* 以同心圆形式可视化 ✔
* 将可视化与医院的照片对齐（验证同心圆的方向，验证数据处理正确性）X

### 读取数据和画图接口

In [2]:
def load_data(dir):
    data = []
    # 打开文件
    with open(dir, 'r') as file:
        # 逐行读取文件内容
        for line in file:
            # 使用split()方法按空格分割每一行数据，得到一个浮点数列表
            float_values = [float(value) for value in line.split()]
            # 将该行数据添加到总数据列表中
            data.append(float_values)
    return data




In [3]:
def visualize_matrix(matrix_data):
    # 使用Seaborn绘制热力图
    plt.figure(figsize=(12, 8))  # 设置图形大小
    sns.heatmap(matrix_data, cmap='viridis')  # 使用viridis颜色映射
    plt.title('Heatmap of Matrix Data')  # 设置标题
    plt.xlabel('Columns')  # 设置x轴标签
    plt.ylabel('Rows')  # 设置y轴标签
    plt.show()

In [4]:
def visualize_circle(data):
  
    # 生成一些示例数据，这里假设你的数据是一个长度为30的列表
    data = data.T   #每行是一个同心圆
    mvalue=0
            
            
    
    
    data = data-data.min()
    
    
    
    data = data/data.max()
   

    # 计算同心圆的半径
    max_radius = 10  # 最大半径
    radius_step = max_radius / len(data)  # 半径步长

    # 创建画布和子图
    fig, ax = plt.subplots(figsize=(20, 20))
    ax.set_aspect('equal')

    # 绘制同心圆热力图
    x_list = []
    y_list = []
    c_list = []
    for circle_id in range(len(data)):
        theta = np.linspace(0, 2 * np.pi, data.shape[1])  # 以数据的长度生成角度范围
        r = circle_id * radius_step  # 当前半径
        x = r * np.cos(theta)  # x坐标
        y = r * np.sin(theta)  # y坐标
        #colors = plt.cm.hot((data[circle_id])/10)  # 根据数据值选择颜色
        colors = data[circle_id]
        
        #colors = plt.cm.hot(np.linspace(0, 1, len(theta)))  # 每个点的颜色
        #for i in range(len(x)):
        #ax.scatter(x, y, c=colors,cmap='viridis')
        x_list.append(x)
        y_list.append(y)
        c_list.append(colors)

    x_list = np.concatenate(x_list,axis=0)
    y_list = np.concatenate(y_list,axis=0)
    c_list = np.concatenate(c_list,axis=0)

    scatter = ax.scatter(x_list, y_list, c=c_list,cmap='viridis')

    # 添加颜色条
    cbar = fig.colorbar(scatter, ax=ax)
    # 设置坐标轴
    ax.set_xlim(-max_radius, max_radius)
    ax.set_ylim(-max_radius, max_radius)
    ax.axis('off')  # 关闭坐标轴

    # 显示图形
    #plt.savefig('axl.png',dpi=300)
    plt.show()

### axl-轴向曲率数据(单位:mm)

In [ ]:

dir = '/Users/txh/Desktop/research/lwk/ok/6.0_after_filtering/1/period2_0001yukaize_Left_2022-01-8_14-05-36.tgl'
origin_data = load_data(dir)
# 统计各行的长度
length_data = np.zeros(len(origin_data)).astype(np.int32)
for i,line in enumerate(origin_data):
    length_data[i] = len(line)
max_length = max(length_data)   # 最大长度
# 填补数据
pad_data = np.zeros((len(origin_data),max_length),dtype=np.float32)
for i,line in enumerate(origin_data):
    pad_data[i,:len(line)] = np.array(line)

visualize_circle(pad_data)

### tgl-切向曲率数据(单位:mm)

In [7]:
def Find_vacancy_insert(data, l):
    #对于数据中的非零值进行转化
    for i in range (data.shape[0]):
        for j in range (data.shape[1]):
            if data[i][j]>0.1:
                data[i][j]=337.5/data[i][j]     
    #模拟环形数据结构 扩充线性列表使头尾紧邻 
    data_tail=data[:,-l:]
    data_head=data[:,0:l]
    data=np.concatenate((data_tail,data,data_head,),axis=1)
    lenth = data.shape[0]
    #对整个数组完成插值 逻辑是对于给定长度参数l 只对连续不多于l个为0的数据进行线性插值
    for i in range(lenth):
        tmp1 = -1
        vacant_number = 0           
        for j in range(len(data[i])):
            if data[i, j] == 0 and vacant_number == 0:
                tmp1 = j
                vacant_number += 1
            elif data[i,j]==0 and vacant_number!=0: vacant_number+=1
                
            elif data[i, j] != 0 and vacant_number <= l and vacant_number >= 1:
                data[i, tmp1:j] = np.nan
                vacant_number = 0
            elif data[i, j] != 0 and vacant_number > l:
                vacant_number = 0       
    
    for i in range(data.shape[0]):
        row = data[i]
        nan_indices = np.where(np.isnan(row))[0]
        valid_indices = np.where(~np.isnan(row))[0]
        if len(nan_indices) > 0:
            f = interp1d(valid_indices, row[valid_indices], kind='linear', fill_value="extrapolate")
            data[i, nan_indices] = f(nan_indices)    
    return data[:,l:-l]

In [ ]:

dir = '/Users/txh/Desktop/research/lwk/ok/6.0_after_filtering/1/period2_0001yukaize_Left_2022-01-8_14-05-36.tgl'
origin_data = load_data(dir)
# 统计各行的长度
length_data = np.zeros(len(origin_data)).astype(np.int32)
for i,line in enumerate(origin_data):
    length_data[i] = len(line)
max_length = max(length_data)   # 最大长度
# 填补数据
pad_data = np.zeros((len(origin_data),max_length),dtype=np.float32)
for i,line in enumerate(origin_data):
    pad_data[i,:len(line)] = np.array(line)
    
tmp=[]
tmp=Find_vacancy_insert(pad_data.copy().T, 18)
tmp=tmp.T

c=tmp.copy()

for i in range (pad_data.shape[0]):
        for j in range (pad_data.shape[1]):
            if pad_data[i][j]>0.1:
                pad_data[i][j]=337.5/pad_data[i][j]  
visualize_circle(pad_data)

visualize_circle(c)

### hgt-角膜高度数据(单位:mm)

In [ ]:

dir = 'Data/Data/5.8/0001yukaize/L/0/0001yukaize_Left_2023-1-15_10-12-45.hgt'
origin_data = load_data(dir)
# 统计各行的长度
length_data = np.zeros(len(origin_data)).astype(np.int32)
for i,line in enumerate(origin_data):
    length_data[i] = len(line)
max_length = max(length_data)   # 最大长度
# 填补数据
pad_data = np.zeros((len(origin_data),max_length),dtype=np.float32)
for i,line in enumerate(origin_data):
    pad_data[i,:len(line)] = np.array(line)

visualize_circle(pad_data)

### dst-径向距离数据(单位:mm)


In [ ]:

dir = 'Data/Data/5.8/0001yukaize/L/0/0001yukaize_Left_2023-1-15_10-12-45.dst'
origin_data = load_data(dir)
# 统计各行的长度
length_data = np.zeros(len(origin_data)).astype(np.int32)
for i,line in enumerate(origin_data):
    length_data[i] = len(line)
max_length = max(length_data)   # 最大长度
# 填补数据
pad_data = np.zeros((len(origin_data),max_length),dtype=np.float32)
for i,line in enumerate(origin_data):
    pad_data[i,:len(line)] = np.array(line)

visualize_circle(pad_data)

### slp-角膜斜度数据

In [ ]:

dir = 'Data/Data/5.8/0001yukaize/L/0/0001yukaize_Left_2023-1-15_10-12-45.slp'
origin_data = load_data(dir)
# 统计各行的长度
length_data = np.zeros(len(origin_data)).astype(np.int32)
for i,line in enumerate(origin_data):
    length_data[i] = len(line)
max_length = max(length_data)   # 最大长度
# 填补数据
pad_data = np.zeros((len(origin_data),max_length),dtype=np.float32)
for i,line in enumerate(origin_data):
    pad_data[i,:len(line)] = np.array(line)

visualize_circle(pad_data)